In [2]:
from pytorch_pretrained_bert import OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer
import torch
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>']}
MODEL_INPUTS = ["input_ids", "mc_token_ids", "lm_labels", "mc_labels", "token_type_ids"]
PADDED_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]

def add_special_tokens_(model, tokenizer):
    """ Add special tokens to the tokenizer and the model if they have not already been added. """
    num_added_tokens = tokenizer.set_special_tokens(SPECIAL_TOKENS) # doesn't add if they are already there
    model.set_num_special_tokens(len(SPECIAL_TOKENS))

tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
add_special_tokens_(model, tokenizer)

D = torch.load('dataloader.pyobj')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [5]:
lm_labels[0]

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  

In [1]:
lm_labels[ lm_labels == -100] = -1
lm_labels[0]

NameError: name 'lm_labels' is not defined

In [4]:
batch = next(iter(D))
input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch
#lm_labels[ lm_labels == -100] = -1

#lm_labels -100
(lm_loss), (mc_loss), *_ = model( input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids, mc_labels=mc_labels, lm_labels=lm_labels )
#lm_loss, mc_loss = model(input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids)
#lm_loss, mc_loss = model(input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids)
# Total loss as a weighted sum
lm_coef, mc_coef = 2.0,1.0
total_loss = lm_loss * lm_coef + mc_loss * mc_coef


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:97

In [44]:
total_loss

tensor(13.4934, grad_fn=<AddBackward0>)

In [1]:
from pytorch_pretrained_bert import OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer
# What is merges
# https://neural-monkey.readthedocs.io/en/latest/machine_translation.html
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [5]:
# We will use 5 special tokens:
# - <bos> to indicate the start of the sequence
# - <eos> to indicate the end of the sequence
# - <speaker1> to indicate the beginning and the tokens of an utterance from the user
# - <speaker2> to indicate the beginning and the tokens of an utterance from the bot
# - <pad> as a padding token to build batches of sequences
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
bos, eos, speaker1, speaker2, _pad = SPECIAL_TOKENS
# We can add these special tokens to the vocabulary and the embeddings of the model:

#tokenizer.set_special_tokens(SPECIAL_TOKENS)
#model.set_num_special_tokens(len(SPECIAL_TOKENS))

In [11]:
from itertools import chain
def list_cat(l): return list(chain(*l))

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    #def spkr_tkn(i): return [speaker2 if (len(sequence)-i) % 2 else speaker1]
    def spkr(i): return speaker1 if (i+1) % 2 else speaker2
    persona  = [[bos] + list_cat(persona)]
    history = [[spkr(i)] + chat for i,chat in enumerate(history)]
    reply = [[spkr(len(history))] + reply + [eos]]
    sequence = persona + history + reply
    
    # Build our word, segments and position inputs from the sequence
    words = list_cat(sequence)                          # word tokens
    #segments = [speaker2 if i % 2 else speaker1             # segment tokens
                #for i, s in enumerate(sequence) for _ in s]
    segments = [ [spkr(i)]*len(chat) for i,chat in enumerate(sequence) ]
    segments = list_cat(segments)
    
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

# Let's define our contexts and special tokens
persona = [["i", "like", "playing", "football", "."],
           ["i", "am", "from", "NYC", "."]]
history = [["hello", "how", "are", "you", "?"],
           ["i", "am", "fine", "thanks", "."]]
reply = ["great", "to", "hear"]
words, segments, position, sequence = build_inputs(persona, history, reply)
words = tokenizer.convert_tokens_to_ids(words)
segments = tokenizer.convert_tokens_to_ids(segments)
words

[40478,
 11,
 14594,
 0,
 0,
 1,
 11,
 1574,
 0,
 0,
 1,
 40480,
 0,
 1991,
 2183,
 7159,
 19,
 40481,
 11,
 1574,
 0,
 12389,
 1,
 40480,
 5201,
 571,
 863,
 40479]

In [12]:
# Let's add a distractor to our previously defined persona, history and reply
distractor = ["sorry", "to", "hear", "that"]
# Build & tokenize inputs ending with our distractor like we did with the gold reply
words_distractor, segments_distractor, _, _ = build_inputs(persona, history, distractor)
words_distractor = tokenizer.convert_tokens_to_ids(words_distractor)
segments_distractor = tokenizer.convert_tokens_to_ids(segments_distractor)

In [13]:
import torch
# Prepare our language modeling targets: keep only the reply segment, -1 on the rest
#lm_targets = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])
r = len(reply)-1
A,B = words[:-r], words[-r:]
lm_targets = [-1]*len(A) +B
lm_distractor = [-1] * len(words_distractor)

# Store the position of the last tokens for the next-sentence prediction loss
last_token = len(words) - 1
last_token_distractor = len(words_distractor) - 1

# Now we can pad reply and distractor inputs and targets to the same length
L = max(len(words), len(words_distractor))
def pad(x, pad_tkn, L): return x + [pad_tkn] * (L - len(x))


pad_tkn = tokenizer.convert_tokens_to_ids('<pad>')
words, words_distractor = pad(words, pad_tkn, L), pad(words_distractor, pad_tkn, L)
segments, segments_distractor  = pad(segments, pad_tkn, L), pad(segments_distractor, pad_tkn, L)
pad_tkn = -1
lm_targets, lm_distractor = pad(lm_targets,pad_tkn,L), pad(lm_distractor,pad_tkn,L)
 
input_ids = torch.tensor([[words, words_distractor]]).long()
token_type_ids = torch.tensor([[segments, segments_distractor]]).long()
# Positions tokens can be automatically created by the model as (0, 1, ..., N)
# Last tokens location
mc_token_ids = torch.tensor([[last_token, last_token_distractor]]).long()
# Language modeling labels
lm_labels = torch.tensor([[lm_targets, lm_distractor]]).long()
# Next-sentence prediction labels
mc_labels = torch.tensor([0]).long()

In [14]:
#input_ids = 'input word index tokens'
#token_type_ids = 'masks of speaker1 and speaker2'
#mc_token_ids = 'idx of last token input'
#lm_labels = 'lm_masks of the last reply'
#mc_labels = 'multiple choice of distractor and non distractor'

In [15]:
# Forward pass

lm_loss, mc_loss = model(input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids)
# Total loss as a weighted sum
lm_coef, mc_coef = 2.0,1.0
total_loss = lm_loss * lm_coef + mc_loss * mc_coef
#lm, mc

In [116]:
#language modeling
#multiple choice

tensor(18.3471, grad_fn=<AddBackward0>)

In [49]:
import json
from pytorch_pretrained_bert import cached_path

#url = "s3://datasets.huggingface.co/personachat/personachat_self_original.json"
url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"
# Download and load JSON dataset
#personachat_file = cached_path(url)
personachat_file = 'convai_data.json'
with open(personachat_file, "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())

# Tokenize and encode the dataset using our loaded GPT tokenizer
def tokenize(obj):
    if isinstance(obj, str):
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
    if isinstance(obj, dict):
        return dict((n, tokenize(o)) for n, o in obj.items())
    return list(tokenize(o) for o in obj)
 
#dataset = tokenize(dataset)


In [73]:
d = dataset['train'][2136]
d.keys()
#d['personality']
for ut in d['utterances']:
    #ut = d['utterances'][2]
    print( len(ut['candidates']) )
    #print('----------------------------------------------')
    #print( ut['history'])
    #print(ut['candidates'][-1] )

20
20
20
20
20
20
20


In [35]:
import torch
#torch.save(dataset, 'convai_data')
#{'train':[{'personality':[], 'utterances':[]},], 'valid':None}
D = torch.load('convai_data.tkn')

In [48]:
d = D['train'][0]
print( d.keys() )
print( 'personality', d['personality'] )
#print( 'utterances', d['utterances'][1] .keys())
print( 'utterances', d['utterances'][2] .keys())

dict_keys(['personality', 'utterances'])
personality [[249, 649, 485, 5371, 4621, 7654, 239], [249, 649, 485, 799, 5408, 239], [249, 649, 485, 4008, 246, 2877, 239], [547, 3898, 6974, 544, 13537, 239]]
utterances dict_keys(['candidates', 'history'])


In [38]:
pad_tkn = tokenizer.convert_tokens_to_ids('<pad>')
pad_tkn

40482